In [1]:
import sys
import sklearn
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import os
from functools import partial

np.random.seed(42)
tf.random.set_seed(42)

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

## mnist preprocessor

In [6]:
def preprocessor(data, valid_nums = 5000):
    # train/valid/test split & scaling
    (X_train_full, y_train_full), (X_test, y_test) = data
    X_train_full = X_train_full / 255.
    X_test = X_test / 255.
    X_train, X_valid = X_train_full[:-(valid_nums)], X_train_full[-(valid_nums):]
    y_train, y_valid = y_train_full[:-(valid_nums)], y_train_full[-(valid_nums):]

    # [28*28] -> [28*28*1]
    X_train = X_train[..., np.newaxis]
    X_valid = X_valid[..., np.newaxis]
    X_test = X_test[..., np.newaxis]
    
    return X_train, y_train, X_valid, y_valid, X_test, y_test

## model body

In [38]:
def middle_block(x, conv_nums, growth_rate):
        for i in range(1, conv_nums+1):
            x = layers.Conv2D(growth_rate * (2**i), 3, padding = "same")(x)
            x = layers.BatchNormalization()(x)
            x = layers.Activation('relu')(x)
            x = layers.Conv2D(growth_rate * (2**i), 3, padding = "same")(x)
            x = layers.BatchNormalization()(x)
            x = layers.Activation('relu')(x)
            x = keras.layers.MaxPool2D(2)(x)
        return x

def create_NN(conv_nums = 3, dropout_rate = 0.3, growth_rate = 32):
    
    ## Entry Flow
    inputs = keras.layers.Input((28, 28, 1))
    conv_1 = layers.Conv2D(growth_rate, 5, strides = 2, padding = "same")(inputs)
    pool_1 = layers.MaxPooling2D(2)(conv_1) 
    
    ## Middle Flow
    x = middle_block(pool_1, conv_nums, growth_rate)
    
    ## Exit Flow
    flatten = tf.keras.layers.Flatten()(x)
    fc1 = layers.Dense(growth_rate * 4, activation='relu')(flatten)
    fc1 = layers.Dropout(dropout_rate)(fc1)
    fc2 = layers.Dense(growth_rate * 2, activation='relu')(fc1)
    fc2 = layers.Dropout(dropout_rate)(fc2)
    outputs = layers.Dense(10, activation='relu')(fc2)
    
    model = keras.models.Model(inputs = inputs, outputs = outputs)
    
    return model

In [ ]:
# Sequential API
model = keras.models.Sequential([
    keras.layers.Conv2D(32, kernel_size=3, padding="same", activation="relu"),
    keras.layers.Conv2D(64, kernel_size=3, padding="same", activation="relu"),
    keras.layers.MaxPool2D(2),
    
    keras.layers.Flatten(),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])

model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))
model.evaluate(X_test, y_test)


## loss & optimizer define

# Training

In [ ]:
# data loading
mnist = keras.datasets.mnist.load_data()
X_train, y_train, X_valid, y_valid, X_test, y_test = preprocessor(data = mnist)

# model create
model = create_NN(conv_nums = 2)

# fit / evaluate
model.compile(loss="sparse_categorical_crossentropy", optimizer = "nadam",
              metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
model.evaluate(X_test, y_test)